In [119]:
# Import required libraries
from datetime import datetime as dt
import os
from pathlib import Path
import pandas as pd
from pandas import DatetimeIndex
import hvplot.pandas
import sqlalchemy
from sqlalchemy import inspect
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import holoviews as hv

In [103]:
# Select date frame you would like to take into account for analysis
first_date = '2021-01-01'
last_date = '2022-12-31'

### Finished

In [169]:
# Create hypothetical temperature increase
hypo_temp_increase = input("how much do you think temperature will increase?")

# set hypo temp increase as floating 
hypo_temp_increase = float(hypo_temp_increase)

### Finished

how much do you think temperature will increase? 2


In [170]:
# Create list with all seasson per year
season_list = ['winter','spring','fall','summer']

### Finished

In [171]:
# Import Weather Data get csv weather data and format 
weather_df = pd.read_csv(Path(
    './/Resources/weather_newyorkny_download.csv')
)

### Finished

In [172]:
# CLEAN WEATHER DATA 

try:
    #strip left 10 characters from weather data periods.dateTimeISO date to create date column
    weather_df['date'] = weather_df['periods.dateTimeISO'].str[:10]

    # change date format to date with actual date from string for easier plotting and use of date column
    weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date

    # set date as index in weather data
    weather_df = weather_df.set_index('date')

    # remove unnamed column
    weather_df.drop(columns=["Unnamed: 0"],inplace=True)
    weather_df.drop(columns=["periods.dateTimeISO"],inplace=True) 

except:
    pass
    print('You ran this part already, no worries check if below dataframe is correctly configured')
    
# Display Weather Data
weather_df.head()

### Finished

,place.name,place.country,periods.tempF,periods.feelslikeF,periods.humidity
date,,,,,
2021-01-01,new york,us,39.94,39.94,62
2021-01-02,new york,us,52.54,52.54,62
2021-01-03,new york,us,37.80,31.51,86
2021-01-04,new york,us,44.39,42.70,67
2021-01-05,new york,us,40.32,40.32,69


In [173]:
# Import Energy Usage Data
energy_df = pd.read_csv(Path(
    './/Resources/Energy_usage_per_day_2021_2022.csv')
)

### Finished

In [174]:
# CLEAN ENERGY USAGE DATA 

try:
    #create new column date seperating start date column into date and time
    energy_df['date'] = pd.to_datetime(energy_df['Start Date']).dt.date
    
    # set date as index in weather data
    energy_df = energy_df.set_index('date')
    
    # Remove orignial start date from data frame (not necessary
    energy_df.drop(columns=['Start Date'], inplace=True)

except:
    pass
    print('You already ran this part already, no worries check if below dataframe is correctly configured')


# Display Weather Data
energy_df.head()


### Finished

,Meter,kWh
date,,
2021-01-01,Meter #98271584,65.50
2021-01-02,Meter #98271584,74.20
2021-01-03,Meter #98271584,76.46
2021-01-04,Meter #98271584,59.16
2021-01-05,Meter #98271584,60.90


In [175]:
# Merge both Energy and Weather dataframe

# select timeframes to analyze
start_date = pd.to_datetime(first_date).date()
end_date = pd.to_datetime(last_date).date()

#select only data within set time frame from weather df
weather_df_select = weather_df.loc[start_date:end_date]

#select only data within set time frame from Energy df
energy_df_select = energy_df.loc[start_date:end_date]

# merge weather data with the energy usage data
merged_data = pd.merge(weather_df_select,energy_df_select, how = 'inner', on = 'date')

# Add columns day, month, year and month_date to be able to make interactive graphs later (month_date is used to determine which season it is in next step)
merged_data['day'] = pd.DatetimeIndex(merged_data.index).day
merged_data['month'] = pd.DatetimeIndex(merged_data.index).month
merged_data['year'] = pd.DatetimeIndex(merged_data.index).year
merged_data['month_date'] = pd.DatetimeIndex(merged_data.index).strftime('%m%d')
merged_data['month_date'] = merged_data['month_date'].astype(float)

# create a column that displays the season (winter, spring, fall or summer)
merged_data['season'] = np.where((merged_data['month_date'] >= 1221),"winter",
                                np.where((merged_data['month_date'] >=101) & (merged_data['month_date'] <321), "winter",
                                         np.where((merged_data['month_date'] >=321) & (merged_data['month_date'] <621), "spring",
                                                  np.where((merged_data['month_date'] >=621) & (merged_data['month_date'] <921), "summer","fall"))))

### Finished

In [176]:
# include temperature increase
merged_data['hypo_temp'] = merged_data['periods.tempF'] + hypo_temp_increase
merged_data.head()


### Finished

,place.name,place.country,periods.tempF,periods.feelslikeF,periods.humidity,Meter,kWh,day,month,year,month_date,season,hypo_temp
date,,,,,,,,,,,,,
2021-01-01,new york,us,39.94,39.94,62,Meter #98271584,65.50,1,1,2021,101.0,winter,41.94
2021-01-02,new york,us,52.54,52.54,62,Meter #98271584,74.20,2,1,2021,102.0,winter,54.54
2021-01-03,new york,us,37.80,31.51,86,Meter #98271584,76.46,3,1,2021,103.0,winter,39.80
2021-01-04,new york,us,44.39,42.70,67,Meter #98271584,59.16,4,1,2021,104.0,winter,46.39
2021-01-05,new york,us,40.32,40.32,69,Meter #98271584,60.90,5,1,2021,105.0,winter,42.32


In [177]:
# Create the connection string for your SQLite database
database_connection_string = 'sqlite:///'

# Pass the connection string to the SQLAlchemy create_engine function
engine = sqlalchemy.create_engine(database_connection_string)

# Confirm that the database engine was created.
engine

Engine(sqlite:///)

In [178]:
# Using the merged data DataFrame, create a table called weather_kwh_usage inside newly created database
merged_data.to_sql(
    'weather_kwh_usage', 
    engine, 
    index=True, 
    if_exists='replace'
)

# Confirm that the table was created by calling the table_names function
inspect(engine).get_table_names()

['weather_kwh_usage']

In [189]:
# Write the SQL query to return data where season is WINTER
query_winter = f"""
    SELECT *
    FROM weather_kwh_usage
    WHERE season = 'winter'
        """
# Using the query, read the data from the database into a Pandas DataFrame
winter_df = pd.read_sql_query(query_winter, con=engine)

### Finished

In [190]:
# Perform Linear Regression

# Prepare data to perform linear regression for winter
x_winter = np.array(winter_df['periods.tempF']).reshape((-1,1))
y_winter = np.array(winter_df['kWh'])

# Set model parameters
model_winter = LinearRegression().fit(x_winter,y_winter)

# Display Results
r_sq_winter = model_spring.score(x_winter,y_winter)
print(f"coefficient of determination: {r_sq_winter}")
print(f"intercept: {model_winter.intercept_}")
print(f"slope: {model_winter.coef_}")

# determine slope and intercept of regression line
y_intercept_winter = model_winter.intercept_
gradient_winter = model_winter.coef_[0]

slope_winter = hv.Slope(gradient_winter,y_intercept_winter)
scatter_winter = hv.Scatter((x_winter, y_winter))

### Finished

coefficient of determination: -2.8175723945946634
intercept: 114.97182489123604
slope: [-1.06126086]


In [225]:
# Combine scatter and regression line

combined_scatter_winter = scatter_winter * slope_winter
graph_winter = combined_scatter_winter.opts(
    height=500,
    width=500,
    xlabel = 'Temperature',
    ylabel = 'kWh Usage',
    title=f'Usage Scatter with Linear Regression winter Season,\n with slope of {gradient_winter:.2f} and intercept of {y_intercept_winter:.2f}')

### Finished

In [215]:
# Write the SQL query to return data where season is WINTER
query_spring = f"""
    SELECT *
    FROM weather_kwh_usage
    WHERE season = 'spring'
        """
# Using the query, read the data from the database into a Pandas DataFrame
spring_df = pd.read_sql_query(query_spring, con=engine)

### Finished

In [216]:
# Perform Linear Regression

# Prepare data to perform linear regression for spring
x_spring = np.array(spring_df['periods.tempF']).reshape((-1,1))
y_spring = np.array(spring_df['kWh'])

# Set model parameters
model_spring = LinearRegression().fit(x_spring,y_spring)

# Display Results
r_sq_spring = model_spring.score(x_spring,y_spring)
print(f"coefficient of determination: {r_sq_spring}")
print(f"intercept: {model_spring.intercept_}")
print(f"slope: {model_spring.coef_}")

# determine slope and intercept of regression line
y_intercept_spring = model_spring.intercept_
gradient_spring = model_spring.coef_[0]

slope_spring = hv.Slope(gradient_spring,y_intercept_spring)
scatter_spring = hv.Scatter((x_spring, y_spring))

### Finished

coefficient of determination: 0.020865184360761013
intercept: 38.15401956176606
slope: [0.22089584]


In [226]:
# Combine scatter and regression line

combined_scatter_spring = scatter_spring * slope_spring
graph_spring = combined_scatter_spring.opts(
    height=500,
    width=500,
    xlabel = 'Temperature',
    ylabel = 'kWh Usage',
    title=f'Usage Scatter with Linear Regression spring Season,\n with slope of {gradient_spring:.2f} and intercept of {y_intercept_spring:.2f}')

### Finished

In [227]:
# Write the SQL query to return data where season is WINTER
query_summer = f"""
    SELECT *
    FROM weather_kwh_usage
    WHERE season = 'summer'
        """
# Using the query, read the data from the database into a Pandas DataFrame
summer_df = pd.read_sql_query(query_summer, con=engine)

### Finished

In [219]:
# Perform Linear Regression

# Prepare data to perform linear regression for summer
x_summer = np.array(summer_df['periods.tempF']).reshape((-1,1))
y_summer = np.array(summer_df['kWh'])

# Set model parameters
model_summer = LinearRegression().fit(x_summer,y_summer)

# Display Results
r_sq_summer = model_summer.score(x_summer,y_summer)
print(f"coefficient of determination: {r_sq_summer}")
print(f"intercept: {model_summer.intercept_}")
print(f"slope: {model_summer.coef_}")

# determine slope and intercept of regression line
y_intercept_summer = model_summer.intercept_
gradient_summer = model_summer.coef_[0]

slope_summer = hv.Slope(gradient_summer,y_intercept_summer)
scatter_summer = hv.Scatter((x_summer, y_summer))

### Finished

coefficient of determination: 0.0023790107195850174
intercept: 74.07201246989692
slope: [-0.12436121]


In [228]:
# Combine scatter and regression line

combined_scatter_summer = scatter_summer * slope_summer
graph_summer = combined_scatter_summer.opts(
    height=500,
    width=500,
    xlabel = 'Temperature',
    ylabel = 'kWh Usage',
    title=f'Usage Scatter with Linear Regression summer Season,\n with slope of {gradient_summer:.2f} and intercept of {y_intercept_summer:.2f}')

### Finished

In [221]:
# Write the SQL query to return data where season is WINTER
query_fall = f"""
    SELECT *
    FROM weather_kwh_usage
    WHERE season = 'fall'
        """
# Using the query, read the data from the database into a Pandas DataFrame
fall_df = pd.read_sql_query(query_fall, con=engine)

# Create scatter plot for season
fall_scatter = fall_df.hvplot.scatter(
x='kWh',
y='periods.tempF',
title='fall_scatter')

In [222]:
# Perform Linear Regression

# Prepare data to perform linear regression for winter
x_fall = np.array(fall_df['periods.tempF']).reshape((-1,1))
y_fall = np.array(fall_df['kWh'])

# Set model parameters
model_fall = LinearRegression().fit(x_fall,y_fall)

# Display Results
r_sq_fall = model_spring.score(x_fall,y_fall)
print(f"coefficient of determination: {r_sq_fall}")
print(f"intercept: {model_fall.intercept_}")
print(f"slope: {model_fall.coef_}")

# determine slope and intercept of regression line
y_intercept_fall = model_fall.intercept_
gradient_fall = model_fall.coef_[0]

slope_fall = hv.Slope(gradient_fall,y_intercept_fall)
scatter_fall = hv.Scatter((x_fall, y_fall))

### Finished

coefficient of determination: -0.44062404104014297
intercept: 78.34906616125895
slope: [-0.36768012]


In [229]:
# Combine scatter and regression line

combined_scatter_fall = scatter_fall * slope_fall
graph_fall = combined_scatter_fall.opts(
    height=500,
    width=500,
    xlabel = 'Temperature',
    ylabel = 'kWh Usage',
    title=f'Usage Scatter with Linear Regression Fall Season,\n with slope of {gradient_fall:.2f} and intercept of {y_intercept_fall:.2f}')

### Finished

In [232]:
# Create seasonal scatter plot with individual regression lines

winter_spring_scatter = graph_winter + graph_spring
summer_fall_scatter = graph_summer + graph_fall
display(winter_spring_scatter)
display(summer_fall_scatter)

### Finished

:Layout
   .Overlay.I  :Overlay
      .Scatter.I    :Scatter   [x]   (y)
      .Annotation.I :Slope   [x,y]
   .Overlay.II :Overlay
      .Scatter.I    :Scatter   [x]   (y)
      .Annotation.I :Slope   [x,y]

:Layout
   .Overlay.I  :Overlay
      .Scatter.I    :Scatter   [x]   (y)
      .Annotation.I :Slope   [x,y]
   .Overlay.II :Overlay
      .Scatter.I    :Scatter   [x]   (y)
      .Annotation.I :Slope   [x,y]

In [ ]:
#### NOT WORKING, JUST HERE JUST IN CASE
# predict response
y_pred = model.predict(x)
print(f'predicted response:\n{y_pred}')

predicted response:
[ 72.58506632  59.21317955  74.85616456  67.86245552  72.1817872
  69.12535594  70.01681505  75.36556977  75.54598411  73.26427327
  77.88075799  67.74571682  68.99800463  63.73415079  69.50740984
  67.2150864   69.22086941  67.04528466  69.13596854  71.02501287
  73.26427327  68.25512203  78.96324407  80.66126143  76.24641628
  77.75340669  74.76065108  76.44805584  87.26230396  83.73891792
  87.52761917  83.20828749  78.99508189  75.32311933  71.38584156
  70.09110331  69.0829055   79.1330458   77.6472806   79.10120798
  79.70612666  81.91354924  84.01484574  82.34866619  76.5223441
  76.73459627  67.66081596  81.72252229  84.96998051  79.91837884
  76.66030801  76.59663236  78.31587494  72.50016546  61.25080039
  63.63863731  67.76694204  64.63622252  68.35063551  67.86245552
  79.65306362  65.38971772  69.28454506  75.23821846  77.68973104
  74.18757022  74.11328196  51.10514661  59.06460303  45.8837432
  47.2421571   66.94977118  59.4042065   78.53873972  78.67